In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR']= '/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [ ]:
!unzip -q train.zip -d .

In [18]:
%rm -rf dataset

In [19]:
import os
import tensorflow as tf
import shutil
#1.일단 train 안의 모든 파일명을 출력해보고 
#2. 파일명에 cat이 들어있다면 dataset의cat폴더로
#3. 파일명에 dog 들어있다면 dataset의 dog폴더로
os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train'):
  if 'cat' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/cat/'+i) 
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i,'/content/dataset/dog/'+i)    
 


In [69]:
import tensorflow as tf
#애는 데이터 중 80%
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset',
    image_size=(64,64),
    batch_size=64,
    subset='training',
    validation_split=0.2,#20#만큼 검증데이터로 나타남
    seed=1234
)
#애는 데이터중 20%
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset',
    image_size=(64,64),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)


def 전처리함수(i,정답):
  i = tf.cast(i/255.0,tf.float32)
  return i,정답

train_ds=train_ds.map(전처리함수)
val_ds=val_ds.map(전처리함수)

model=tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',input_shape=(64,64,3)),#뒤집어주세요
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    tf.keras.layers.Conv2D(32,(3,3),padding="same",activation='relu'),#color사진이라 input의 마지막은 3
    tf.keras.layers.MaxPooling2D((2,2)),#풀링사이즈 2,2
    tf.keras.layers.Conv2D(64,(3,3),padding="same",activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),#풀링사이즈 2,2
    tf.keras.layers.Dropout(0.2),#overfitting 완화기능(학습용 데이터를 외워버리는걸 방지) 윗레이어의 노드를 일부제거
    tf.keras.layers.Conv2D(128,(3,3),padding="same",activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.summary()


model.compile( loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_9 (RandomFlip)   (None, 64, 64, 3)         0         
_________________________________________________________________
random_rotation_8 (RandomRot (None, 64, 64, 3)         0         
_________________________________________________________________
random_zoom_8 (RandomZoom)   (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_114 (MaxPoolin (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 